In [46]:
!pip install import_ipynb
!pip install h5py

In [0]:
import import_ipynb
import keras
import data_builder
import model_builder
import matplotlib
matplotlib.use("Agg")
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils.np_utils import to_categorical
from keras.models import model_from_json
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow 
import pickle 
import numpy as np
import cv2




In [0]:

fights_video = "/content/drive/My Drive/code/fight-detection-surv-dataset/fights_video"
fights_img = "/content/fights_img"
noFights_img= "/content/noFights_img"
noFights_video="/content/drive/My Drive/code/fight-detection-surv-dataset/noFights_video"
data_dir = "/content"
data_builder.save_img(fights_video, fights_img)
data_builder.save_img(noFights_video, noFights_img)

#sudo apt install kazamdata_dir = "/content"

data_builder.image_builder(data_dir)
 

In [0]:
X = pickle.load(open("X.pickle" , "rb"))
Y = pickle.load(open("Y.pickle" , 'rb'))
y = np.array(Y)

X = X/255.0 
print(len(X))
print(len(y))

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(X, Y,
	test_size=0.25, stratify=Y, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

# initialize the optimizer and model
print("[INFO] compiling model...")
epo = 60
opt = Adam(lr=1e-4, decay=1e-4 / epo)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)
print(trainX.shape)

#build the model
model = model_builder.StridedNet.build(width=160, height=160, depth=1,
	classes= len(lb.classes_), reg=l2(0.0005))




In [5]:
trainX = trainX.reshape(trainX.shape[0], trainX.shape[1], trainX.shape[2], 1)
testX = testX.reshape(testX.shape[0], testX.shape[1], testX.shape[2], 1)

print(trainX.shape)
print(testX.shape)

(12908, 160, 160, 1)
(4303, 160, 160, 1)


In [6]:
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 158, 158, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 77, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 36, 36, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20736)            

In [0]:
# train the network
print("[INFO] training network for {} epochs...".format(epo))
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // 32,
	epochs=60)

In [0]:
classes = ["fights","nofight"]
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=None))

In [0]:
N = epo
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [0]:
# Calling `save('my_model.h5')` creates a h5 file and json file.
#json 
model_json = model.to_json()
with open("/content/drive/My Drive/code/model.json", "w") as json_file:
    json_file.write(model_json)
#HDF5
model.save('/content/drive/My Drive/code/my_h5_model.h5')


In [0]:
# preparation de l'image de test
image_size=160
categorie="fights_img"
test_data = []
path = "/content/drive/My Drive/code/index.jpeg"
image_array = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
new_array = cv2.resize(image_array ,(image_size ,image_size))
test_data.append([new_array , categorie])

        

In [0]:
x = []
y = []
for features , label in test_data:
    x.append(features)
    y.append(label)
x = np.array(x)
x = x/255.0
x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
print(x.shape)
y = lb.fit_transform(y)
print(y)



In [54]:
# load json and create model
json_file = open('/content/drive/My Drive/code/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/code/my_h5_model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(testX, testY, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 93.70%
